### ***Model***  = MLPWithTensorFlowLowLevelAPI.ipynb

In [ ]:
# Definindo a versão do tensorflow:
%tensorflow_version 1.x

In [ ]:
# Importando as bibliotecas necessárias:
import numpy as np
import tensorflow as tf
import graphviz as tfg              # Usar este código! 
from tensorboard import notebook

In [ ]:
# to make this notebook's output stable across runs
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)
    
def shuffle_batch(X, y, batch_size):
    rnd_idx = np.random.permutation(len(X))
    n_batches = len(X) // batch_size
    for batch_idx in np.array_split(rnd_idx, n_batches):
        X_batch, y_batch = X[batch_idx], y[batch_idx]
        yield X_batch, y_batch

In [ ]:
reset_graph()

n_inputs = 28*28 # MNIST
n_hidden1 = 300
n_hidden2 = 100
n_outputs = 10

Dispensa o '***neuron_layer***'

```
def neuron_layer(X, n_neurons, name, activation=None):
    with tf.name_scope(name):
        n_inputs = int(X.get_shape()[1])
        stddev = 2 / np.sqrt(n_inputs)
        init = tf.truncated_normal((n_inputs, n_neurons), stddev=stddev)
        W = tf.Variable(init, name="weights")
        b = tf.Variable(tf.zeros([n_neurons]), name="biases")
        z = tf.matmul(X, W) + b
        if activation=="relu":
            return tf.nn.relu(z)
        else:
            return z
```



In [ ]:
# Importando as bibliotecas para usar o Dense():
from keras.layers import Activation
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
# Configurando modo de compatibilidade com a V1.x
tf.compat.v1.keras.layers.Dense

tensorflow.python.keras.layers.core.Dense

Substituição da função ***neuron_layer***

In [ ]:
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()
X_train = X_train.astype(np.float32).reshape(-1, 28*28) / 255.0
X_test = X_test.astype(np.float32).reshape(-1, 28*28) / 255.0
y_train = y_train.astype(np.int32)
y_test = y_test.astype(np.int32)
X_valid, X_train = X_train[:5000], X_train[5000:]
y_valid, y_train = y_train[:5000], y_train[5000:]

In [ ]:
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n_inputs), name="X")
y = tf.placeholder(tf.int64, shape=(None), name="y")

with tf.name_scope("dnn"):
    hidden1 = tf.layers.dense(X, n_hidden1, name="hidden1")
    hidden2 = tf.layers.dense(hidden1, n_hidden2, name="hidden2",
                              activation=tf.nn.relu)
    logits = tf.layers.dense(hidden2, n_outputs, name="outputs")
    y_proba = tf.nn.softmax(logits)
    
with tf.name_scope("loss"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=y, logits=logits)
    loss = tf.reduce_mean(xentropy, name="loss")

with tf.name_scope("train"):
    optimizer = tf.train.GradientDescentOptimizer(learning_rate)
    training_op = optimizer.minimize(loss)
    
with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
init = tf.global_variables_initializer()
saver = tf.train.Saver()

In [ ]:
n_epochs = 20
n_batches = 50
n_epochs = 40
batch_size = 50

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for X_batch, y_batch in shuffle_batch(X_train, y_train, batch_size):
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_batch = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_valid = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print(epoch, "Batch accuracy:", acc_batch, "Validation accuracy:", acc_valid)

    save_path = saver.save(sess, "./my_model_final.ckpt")

0 Batch accuracy: 0.9 Validation accuracy: 0.9054
1 Batch accuracy: 0.92 Validation accuracy: 0.9244
2 Batch accuracy: 0.94 Validation accuracy: 0.9336
3 Batch accuracy: 0.88 Validation accuracy: 0.9358
4 Batch accuracy: 0.96 Validation accuracy: 0.9436
5 Batch accuracy: 0.94 Validation accuracy: 0.948
6 Batch accuracy: 1.0 Validation accuracy: 0.9542
7 Batch accuracy: 0.94 Validation accuracy: 0.9556
8 Batch accuracy: 0.94 Validation accuracy: 0.956
9 Batch accuracy: 0.94 Validation accuracy: 0.9592
10 Batch accuracy: 0.92 Validation accuracy: 0.961
11 Batch accuracy: 1.0 Validation accuracy: 0.9628
12 Batch accuracy: 0.98 Validation accuracy: 0.9622
13 Batch accuracy: 0.98 Validation accuracy: 0.9648
14 Batch accuracy: 1.0 Validation accuracy: 0.966
15 Batch accuracy: 0.94 Validation accuracy: 0.9668
16 Batch accuracy: 0.98 Validation accuracy: 0.9674
17 Batch accuracy: 1.0 Validation accuracy: 0.9688
18 Batch accuracy: 0.96 Validation accuracy: 0.97
19 Batch accuracy: 0.96 Validatio